## Qué es la convolución?

La convolución es una operación lineal similar a una ecuación lineal, un producto punto o una multiplicación de matrices.

La convolución tiene varias ventajas para analizar imágenes. Como se explicó en el video, preserva la relación entre los elementos y requiere menos parámetros que otros métodos.


Puedes ver la **relación entre los diferentes métodos** que aprendiste:

$$ecuación \ lineal : y = wx + b$$

$$ecuación \ lineal \ con \ múltiples \ variables \ donde \ \mathbf{x} \ es \ un \ vector \ \mathbf{y} = \mathbf{wx} + b$$

$$multiplicación \ de \ matrices \ donde \ \mathbf{X} \ es \ una \ matriz \ \mathbf{y} = \mathbf{wX} + \mathbf{b}$$

$$convolución \ donde \ \mathbf{X} \ y \ \mathbf{Y} \ son \ tensores \ \mathbf{Y} = \mathbf{w} * \mathbf{X} + \mathbf{b}$$

En convolución, el parámetro **w** se denomina kernel. Puedes realizar la convolución en imágenes, donde permites que la variable *image* denote la variable X y *w* denote el parámetro.

<img src="https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0110EN/notebook_images%20/chapter%206/6.1.1xw.png" width="500," align="center">

Cree un objeto de convolución bidimensional utilizando el constructor Conv2d. Los parámetros <code>in_channels</code> y <code>out_channels</code> se utilizarán para esta sección, y el parámetro kernel_size será tres.


In [2]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import numpy as np
from scipy import ndimage, misc

C:\Users\Josue Santana Robled\AppData\Local\Temp\ipykernel_27456\3053585304.py:5: DeprecationWarning: scipy.misc is deprecated and will be removed in 2.0.0
  from scipy import ndimage, misc


In [3]:
conv = nn.Conv2d(in_channels=1, out_channels=1,kernel_size=3)
conv

Conv2d(1, 1, kernel_size=(3, 3), stride=(1, 1))

Dado que los parámetros en <code>nn.Conv2d</code> se inicializan aleatoriamente y se aprenden mediante el entrenamiento, asígnales algunos valores.

In [4]:
conv.state_dict()['weight'][0][0]=torch.tensor([[1.0,0,-1.0],[2.0,0,-2.0],[1.0,0.0,-1.0]])
conv.state_dict()['bias'][0]=0.0
conv.state_dict()

OrderedDict([('weight',
              tensor([[[[ 1.,  0., -1.],
                        [ 2.,  0., -2.],
                        [ 1.,  0., -1.]]]])),
             ('bias', tensor([0.]))])

Cree un tensor ficticio para representar una imagen. La forma de la imagen es (1,1,5,5), donde:

(número de entradas, número de canales, número de filas, número de columnas)

Establezca la tercera columna en 1:

In [5]:
image=torch.zeros(1,1,5,5)
image[0,0,:,2]=1
image

tensor([[[[0., 0., 1., 0., 0.],
          [0., 0., 1., 0., 0.],
          [0., 0., 1., 0., 0.],
          [0., 0., 1., 0., 0.],
          [0., 0., 1., 0., 0.]]]])

Llama al objeto <code>conv</code> en el tensor <code>image</code> como entrada para realizar la convolución y asigna el resultado al tensor <code>z</code>.

In [6]:
z=conv(image)
z

tensor([[[[-4.,  0.,  4.],
          [-4.,  0.,  4.],
          [-4.,  0.,  4.]]]], grad_fn=<ConvolutionBackward0>)

La siguiente animación ilustra el proceso: el núcleo realiza la multiplicación a nivel de elemento en cada elemento de la imagen en la región correspondiente. A continuación, se suman los valores. Después, el núcleo se desplaza y se repite el proceso.

<img src="https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0110EN/notebook_images%20/chapter%206/6.1.1convltuon.gif" width="500," align="center">

### Determinación  del tamaño de la salida

El tamaño de la salida es un parámetro importante. En este laboratorio, se asumirá que las imágenes son cuadradas. Para imágenes rectangulares, se puede utilizar la misma fórmula para cada dimensión de forma independiente.

Sea M el tamaño de la entrada y K el tamaño del núcleo. El tamaño de la salida viene dado por la siguiente fórmula:

$$M_{nueva}=M-K+1$$

Crear un núcleo de tamaño 2:

In [7]:
K=2
conv1 = nn.Conv2d(in_channels=1, out_channels=1,kernel_size=K)
conv1.state_dict()['weight'][0][0]=torch.tensor([[1.0,1.0],[1.0,1.0]])
conv1.state_dict()['bias'][0]=0.0
conv1.state_dict()
conv1

Conv2d(1, 1, kernel_size=(2, 2), stride=(1, 1))

Crea una imagen de tamaño 2:

In [8]:
M=4
image1=torch.ones(1,1,M,M)

<img src="https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0110EN/notebook_images%20/chapter%206/6.1.1kernal2.png" width="500," align="center">

La siguiente ecuación proporciona el resultado:

$$M_{nueva}=M-K+1$$
$$M_{nueva}=4-2+1$$
$$M_{nueva}=3$$

La siguiente animación ilustra el proceso: La primera iteración de la superposición del núcleo de las imágenes produce un resultado. Como el núcleo tiene un tamaño K, hay M-K  elementos para que el núcleo se mueva en dirección horizontal. La misma lógica se aplica a la dirección vertical.

<img src="https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0110EN/notebook_images%20/chapter%206/6.1.1outsize.gif" width="500," align="center">

Realice la convolución y compruebe que el tamaño sea correcto:

In [9]:
z1=conv1(image1)
print("z1:",z1)
print("shape:",z1.shape[2:4])

z1: tensor([[[[4., 4., 4.],
          [4., 4., 4.],
          [4., 4., 4.]]]], grad_fn=<ConvolutionBackward0>)
shape: torch.Size([3, 3])


### Parámetro de Paso

El parámetro stride cambia el número de desplazamientos que realiza el núcleo por iteración. Como resultado, el tamaño de salida también cambia y se calcula mediante la siguiente fórmula:

$$M_{nueva}=\dfrac{M-K}{Paso}+1$$

Crea un objeto de convolución con un paso de 2:

In [10]:
conv3 = nn.Conv2d(in_channels=1, out_channels=1,kernel_size=2,stride=2)

conv3.state_dict()['weight'][0][0]=torch.tensor([[1.0,1.0],[1.0,1.0]])
conv3.state_dict()['bias'][0]=0.0
conv3.state_dict()

OrderedDict([('weight',
              tensor([[[[1., 1.],
                        [1., 1.]]]])),
             ('bias', tensor([0.]))])

Para una imagen con un tamaño de 4, calcule el tamaño de salida:

$$M_{nueva}=\dfrac{M-K}{Paso}+1$$
$$M_{nueva}=\dfrac{4-2}{2}+1$$
$$M_{nueva}=2$$


La siguiente animación ilustra el proceso: La primera iteración de la superposición del núcleo de las imágenes produce un resultado. Dado que el núcleo tiene un tamaño K, hay M-K=2 elementos. El paso es 2 porque se moverán 2 elementos a la vez. Como resultado, se divide M-K por el valor del paso 2:

<img src="https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0110EN/notebook_images%20/chapter%206/6.1.1stride2.gif" width="500," align="center">

Realice la convolución y compruebe que el tamaño sea correcto:

In [11]:
z3=conv3(image1)

print("z3:",z3)
print("shape:",z3.shape[2:4])

z3: tensor([[[[4., 4.],
          [4., 4.]]]], grad_fn=<ConvolutionBackward0>)
shape: torch.Size([2, 2])


### Relleno con ceros

A medida que se aplican convoluciones sucesivas, la imagen se reducirá. Se puede aplicar un relleno con ceros para mantener la imagen en un tamaño razonable, lo que también conserva la información en los bordes.

Además, es posible que no obtenga valores enteros para el tamaño del núcleo. Considere la siguiente imagen:


In [12]:
image1

tensor([[[[1., 1., 1., 1.],
          [1., 1., 1., 1.],
          [1., 1., 1., 1.],
          [1., 1., 1., 1.]]]])

Intenta realizar convoluciones con <code>kernel_size=2</code> y <code>stride=3</code>. Utiliza estos valores:

$$M_{nueva}=\dfrac{M-K}{Paso}+1$$
$$M_{nueva}=\dfrac{4-2}{3}+1$$
$$M_{nueva}=1.666$$

In [13]:
conv4 = nn.Conv2d(in_channels=1, out_channels=1,kernel_size=2,stride=3)
conv4.state_dict()['weight'][0][0]=torch.tensor([[1.0,1.0],[1.0,1.0]])
conv4.state_dict()['bias'][0]=0.0
conv4.state_dict()
z4=conv4(image1)
print("z4:",z4)
print("z4:",z4.shape[2:4])

z4: tensor([[[[4.]]]], grad_fn=<ConvolutionBackward0>)
z4: torch.Size([1, 1])


Puedes añadir filas y columnas de ceros alrededor de la imagen. Esto se denomina relleno. En el constructor <code>Conv2d</code>, se especifica el número de filas o columnas de ceros que se desean añadir con el parámetro padding.

Para una imagen cuadrada, solo tiene que rellenar una columna adicional de ceros en la primera y la última columna. Repita el proceso para las filas. Como resultado, para una imagen cuadrada, la anchura y la altura son el tamaño original más 2 x el número de elementos de relleno especificados. A continuación, puede determinar el tamaño de la salida después de las operaciones posteriores según se muestra en la siguiente ecuación, en la que se determina el tamaño de una imagen después del relleno y, a continuación, se aplica un núcleo de convoluciones de tamaño K.


$$M'=M+2 \times padding$$
$$M_{new}=M'-K+1$$

Considera el siguiente ejemplo:

In [14]:
conv5 = nn.Conv2d(in_channels=1, out_channels=1,kernel_size=2,stride=3,padding=1)

conv5.state_dict()['weight'][0][0]=torch.tensor([[1.0,1.0],[1.0,1.0]])
conv5.state_dict()['bias'][0]=0.0
conv5.state_dict()
z5=conv5(image1)
print("z5:",z5)
print("z5:",z4.shape[2:4])

z5: tensor([[[[1., 2.],
          [2., 4.]]]], grad_fn=<ConvolutionBackward0>)
z5: torch.Size([1, 1])


El proceso es resumido en la siguiente animación:

<img src="https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0110EN/notebook_images%20/chapter%206/6.1.1zeropad.gif" width="500," align="center">

#### Pregunta de Practica

 Se aplica un núcleo de ceros con un tamaño de núcleo = 3 a la siguiente imagen:

In [15]:
Image=torch.randn((1,1,4,4))
Image

tensor([[[[-0.6436, -0.1082, -0.3726, -0.2109],
          [-0.0580,  0.8211, -0.9148,  0.0131],
          [-0.3870, -0.2030, -0.3533,  0.4571],
          [-0.6786,  1.0247, -2.0543, -1.1119]]]])

Pregunta: Sin utilizar la función, determine cuáles son los valores de salida de cada elemento:

Respuesta: Como cada elemento del núcleo es cero, y para cada salida, la imagen se multiplica por el núcleo, el resultado es siempre cero.

Pregunta: Utilice el siguiente objeto de convolución para realizar la convolución en el tensor.

In [16]:
conv = nn.Conv2d(in_channels=1, out_channels=1,kernel_size=3)
conv.state_dict()['weight'][0][0]=torch.tensor([[0,0,0],[0,0,0],[0,0.0,0]])
conv.state_dict()['bias'][0]=0.0

In [17]:
conv(Image)

tensor([[[[0., 0.],
          [0., 0.]]]], grad_fn=<ConvolutionBackward0>)

Pregunta: Tienes una imagen de tamaño 4. Los parámetros son los siguientes:  kernel_size=2, stride=2. ¿Cuál es el tamaño de la salida?

$$(M-K)/Paso +1$$
$$(4-2)/2 +1$$
$$2$$